# Accessing DISP-S1 data using Xarray

In [ ]:
import importlib
from datetime import datetime

from opera_utils import disp

# we need xarray installed
if importlib.util.find_spec("xarray") is None:
    msg = "xarray is required for this example"
    raise ImportError(msg)

products = disp.search(
    frame_id=11116,
    start_datetime=datetime(2020, 1, 1),
    end_datetime=datetime(2020, 12, 31)
)

product_stack = disp.DispProductStack(products)





In [16]:
%%time

from opera_utils.disp.xarray import stack_to_dataarray

da = stack_to_dataarray(product_stack[:5], var="short_wavelength_displacement")

da


CPU times: user 305 ms, sys: 38 ms, total: 343 ms
Wall time: 344 ms


<xarray.DataArray 'short_wavelength_displacement' (time: 5, y: 6898, x: 9618)> Size: 1GB
dask.array<stack, shape=(5, 6898, 9618), dtype=float32, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * time            (time) object 40B 2020-01-04T14:07:34+00:00 ... 2020-02-0...
  * y               (y) float64 55kB 4.114e+06 4.114e+06 ... 3.908e+06 3.908e+06
  * x               (x) float64 77kB 4.998e+05 4.998e+05 ... 7.883e+05 7.883e+05
    reference_date  (time) object 40B 2019-11-05T14:07:36+00:00 ... 2019-11-0...
Attributes:
    crs:        EPSG:32610
    transform:  | 30.00, 0.00, 499800.00|\n| 0.00,-30.00, 4114500.00|\n| 0.00...

In [17]:
data = da[:, 4000:4002, 4000:4002]
data

<xarray.DataArray 'short_wavelength_displacement' (time: 5, y: 2, x: 2)> Size: 80B
dask.array<getitem, shape=(5, 2, 2), dtype=float32, chunksize=(1, 2, 2), chunktype=numpy.ndarray>
Coordinates:
  * time            (time) object 40B 2020-01-04T14:07:34+00:00 ... 2020-02-0...
  * y               (y) float64 16B 3.994e+06 3.994e+06
  * x               (x) float64 16B 6.198e+05 6.198e+05
    reference_date  (time) object 40B 2019-11-05T14:07:36+00:00 ... 2019-11-0...
Attributes:
    crs:        EPSG:32610
    transform:  | 30.00, 0.00, 499800.00|\n| 0.00,-30.00, 4114500.00|\n| 0.00...

In [ ]:
%%time

data.compute()

An alternative way to open the data is to use the `open_file` method, which leverages `fsspec`:

In [ ]:
%%time

open_files = [disp.open_file(p.filename) for p in product_stack[:5]]

CPU times: user 49.4 ms, sys: 19.4 ms, total: 68.8 ms
Wall time: 8.6 s


In [ ]:
print(open_files[0])

<File-like object HTTPFileSystem, https://datapool.asf.alaska.edu/DISP/OPERA-S1/OPERA_L3_DISP-S1_IW_F11116_VV_20191105T140736Z_20200104T140734Z_v1.0_20250410T091243Z.nc>


These open files can be used with `xr.open_dataset` to read the data.